# Logging

1. Motivación
2. sys.stdout y sys.stderr
3. Loguru
    - log levels en Loguru.
    - Personalización de Loguru.
    - Formato de logs.
    - Agregar datos contextuales al logger.
    - Errores con Loguru.
    - Guardar logs en un archivo.


## Motivación

Logging se refiere a la acción de llevar un registro, en algún sitio de algunos eventos que ocurren durante el tiempo de ejecución. 

Registrar los datos correctos en los lugares correctos, nos ayudará a:

- **Depurar errores**: LLevar un registro de los fallos junto con los parámetros que ocasionaron el fallo.
<br>

- **Analizar el rendimiento**: Así podremos planificar la escalabilidad correcta y optimizar costos.
<br>

- **Analizar patrones**: Entender el uso de nuestra aplicación.
<br>

## stdout y stderr


Una primera aproximación podría ser con la función `print`

In [1]:
file_name = 'ejemplo.txt'
with open(file_name, 'r') as f:
    print(f"Abriendo archivo {file_name}.")
    r = f.read()
print(f"Archivo {file_name} cerrado.")
    

Abriendo archivo ejemplo.txt.
Archivo ejemplo.txt cerrado.


Por defecto todos los datos que genera Python pueden estar en dos sitios:

- `stdout`: Lugar en donde se almacenaran temporalmente los output the ejecuciones **sin** errores.

- `stderr`: Lugar en donde se almacenaran temporalmente los output the ejecuciones **con** errores.

En el caso de los notebooks se usa un Stream, y no un archivo.

In [2]:
import sys
print(sys.stderr)

En el caso de una instancia normal de Python el `stderr` luce así:

```Python
import sys
sys.stderr
<_io.TextIOWrapper name='<stderr>' mode='w' encoding='utf-8'>
```

Pero una vez que se ejecuta el programa, no tenemos manera de recuperar los datos. Tampoco tenemos información contextual de la ejecución, e.g. la hora y el día.

## Loguru

Es un paquete para gestionar lo logs en Python, es probablemente el paquete [más utilizado](https://github.com/Delgan/loguru) dentro de su categoría. 

Esto se debe a que su objetivo es facilitar el proceso de configuración de un sistema de logging, proporciona una alternativa mucho más fácil de configurar al módulo de logging predeterminado de Python.

al ser una librería de terceros significa que lo tendremos que installar, **si tenemos un entorno virtual activado de poetry** podemos:

```shell
poetry add loguru
```

Obtendremos algo similar a:

```shell
Using version ^0.6.0 for loguru

Updating dependencies
Resolving dependencies... (0.9s)

Writing lock file

Package operations: 1 install, 0 updates, 0 removals

  • Installing loguru (0.6.0)
```

La forma más básica de usar `loguru` es importando el objeto `logger` del paquete `loguru`. El `logger` está preconfigurado, así que lo podemos usar directamente.


In [4]:
from loguru import logger
logger

<loguru.logger handlers=[(id=0, level=10, sink=stderr)]>

Podemos usar El método `debug()` para registrar un mensaje de nivel DEBUG.

> Debug significa depurar un programa para identificar y corregir errores.

In [5]:
logger.debug("Mensaje de depuración")

2023-01-23 18:53:12.583 | DEBUG    | __main__:<module>:1 - Mensaje de depuración


La salida contiene los siguientes detalles:

- `2023-01-15 19:21:23.866` - `timestamp`, el tiempo justo en que se ejecutó el comando.

- `DEBUG` - el nivel de logging, Esta variable se utiliza para describir el nivel de gravedad.

- `__main__:<módulo>:1` - la ubicación del archivo, el alcance y el número de línea. En este ejemplo:
    - La ubicación del archivo es `__main__`, porque estamos ejecutando el archivo desde el script de entrada de python (notebook)
    - El alcance es `\<module\>` porque el registrador no está ubicado dentro de una clase o una función.
    - La línea es `1` porque el logger está en la primera línea de la celda.
    

### log levels en Loguru.


Los *log levels* (niveles de registro) especifican la gravedad de un log (registro) para que los mensajes se puedan filtrar o priorizar en función de su urgencia.

Loguru ofrece siete niveles de registro:

- `TRACE` (5): se utiliza para registrar información detallada sobre la ruta de ejecución del programa con fines de diagnóstico.
<br>

- `DEBUG` (10): utilizado por los desarrolladores para registrar mensajes con fines de depuración.
<br>

- `INFO` (20): se utiliza para registrar mensajes informativos que describen el funcionamiento normal del programa.
<br>

- `SUCCESS` (25): similar a INFO pero se utiliza para indicar el éxito de una operación.
<br>

- `WARNING` (30): se utiliza para indicar un evento inusual que puede requerir una mayor investigación.
<br>

- `ERROR` (40): se utiliza para registrar condiciones de error que afectaron una operación específica.
<br>

- `CRITICAL` (50): se utiliza para registrar condiciones de error que impiden el funcionamiento de una función principal.
<br>



In [6]:
logger.trace("trace.")
logger.debug("debug.")
logger.info("info")
logger.success("success.")
logger.warning("warning.")
logger.error("error.")
logger.critical("critical.")

2023-01-23 18:56:59.198 | DEBUG    | __main__:<module>:2 - debug.
2023-01-23 18:56:59.200 | INFO     | __main__:<module>:3 - info
2023-01-23 18:56:59.201 | SUCCESS  | __main__:<module>:4 - success.
2023-01-23 18:56:59.203 | WARNING  | __main__:<module>:5 - warning.
2023-01-23 18:56:59.203 | ERROR    | __main__:<module>:6 - error.
2023-01-23 18:56:59.205 | CRITICAL | __main__:<module>:7 - critical.


### Personalización de Loguru.

Loguru simplifica el proceso de configuración utilizando únicamente su método `add()`, Algunos de sus parámetros son:

- `sink`: Especifica un destino para cada registro producido por el registrador. De forma predeterminada, se establece en sys.stderr.
<br>

- `level`: Especifica el nivel de registro mínimo para el registrador.
<br>

- `format`: Útil para definir un formato personalizado para los logs.
<br>

### Formato de logs.

Se puede reformatear los logs a través del parámetro `format` del método `add()`. 

Cada log es un diccionario de Python, que contiene datos como `timestamp`, nivel de registro ,etc. Recomiendo mirar la [documentación](https://loguru.readthedocs.io/en/stable/api/logger.html#record) para ver una lista de todas las directivas de formato.



In [7]:
import sys
from loguru import logger

handler_id = logger.add(sys.stderr, format="{time} | {level} | {message}")

logger.debug("Mensaje")

2023-01-23 19:00:58.023 | DEBUG    | __main__:<module>:6 - Mensaje
2023-01-23T19:00:58.023450+0100 | DEBUG | Mensaje


¿Qué ha sucedido? Resulta que hemos añadido un nuevo `handler` y por tanto cada vez que loggeamos un mensaje estaremos utilizando los `handlers` disponibles

In [8]:
logger.remove(0)
logger.debug("Mensaje")

2023-01-23T19:01:46.225533+0100 | DEBUG | Mensaje


In [9]:
logger.remove(handler_id)
handler_id = logger.add(sys.stderr, format="{time} | {level} | {file} |{message}")
logger.debug("Mensaje")

2023-01-23T19:01:56.065041+0100 | DEBUG | 182477577.py |Mensaje


### Agregar datos contextuales al logger.

Además del mensaje, a menudo es necesario incluir otra información relevante en la entrada de log, e.g. los parámetros de una función o la configuración de nuestra aplicación.

La directiva de formato `{extra}` sirve justo para eso:

In [10]:
import sys
from loguru import logger


class Estimator():
    
    def __init__(self,):
        
        logger.remove()
        logger.add(sys.stderr, format="{time:MMMM D, YYYY > HH:mm:ss} | {level} | {message} | {extra}")

    def predict(self, x: float) -> float:
        y = x*0.1 + 3
        predict_logger = logger.bind(x=x, y=y)
        predict_logger.success("Predicción generada con éxito")
        return y


estimator = Estimator()
estimator.predict(5)

January 23, 2023 > 19:05:44 | SUCCESS | Predicción generada con éxito | {'x': 5, 'y': 3.5}


3.5

In [11]:
estimator.predict(8)

January 23, 2023 > 19:09:48 | SUCCESS | Predicción generada con éxito | {'x': 8, 'y': 3.8}


3.8

### Errores con Loguru.

Los logs son una excelente forma para entender y solucionar errores.

Puede loggear errores automáticamente a medida que ocurren dentro de una función:

In [12]:
logger.remove()
logger.add(sys.stderr, format="{time:MMMM D, YYYY > HH:mm:ss} | {level} | {message} | {extra}")

@logger.catch
def test(x):
    50/x

test(0)

January 23, 2023 > 19:10:49 | ERROR | An error has been caught in function '<module>', process 'MainProcess' (1897), thread 'MainThread' (4784453120): | {}
Traceback (most recent call last):

  File "/Users/heber.trujilloglovoapp.com/.pyenv/versions/3.8.6/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
           │         │     └ {'__name__': '__main__', '__doc__': 'Entry point for launching an IPython kernel.\n\nThis is separate from the ipykernel pack...
           │         └ <code object <module> at 0x10ebc49d0, file "/Users/heber.trujilloglovoapp.com/projects/curso-python-cac/.venv/lib/python3.8/s...
           └ <function _run_code at 0x10ebc5280>
  File "/Users/heber.trujilloglovoapp.com/.pyenv/versions/3.8.6/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
         │     └ {'__name__': '__main__', '__doc__': 'Entry point for launching an IPython kernel.\n\nThis is separate from the ipykernel pac

### Guardar logs en un archivo.

La opción de `sink` del método `add()` permite elegir el destino de todos los log emitidos a través de un logger. 

Hasta ahora, solo hemos sólo hemos usado `stderr`, pero también podemos crear un archivo. El archivo debe tener la extensión *.log

In [ ]:
logger.add("loguru.log")

logger.debug("Mensaje debug.")